In [ ]:
!pip install PyDrive

In [1]:
import os
import mykeys

project_name = 'CH15_Evaluation'

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = project_name
os.environ["LANGCHAIN_API_KEY"] = mykeys.get_key('LANG')
os.environ["LANGCHAIN_HUB_API_KEY"] = mykeys.get_key('LANG')
os.environ["OPENAI_API_KEY"] = mykeys.get_key('GPT')
os.environ["GOOGLE_API_KEY"] = mykeys.get_key('GOO')
os.environ["HUGGINGFACEHUB_API_TOKEN"] = mykeys.get_key('HF')
os.environ["UPSTAGE_API_KEY"] = mykeys.get_key('UP')
os.environ["COHERE_API_KEY"] = mykeys.get_key('COH')
os.environ["JINA_API_KEY"] = mykeys.get_key('JINA')
os.environ["ANTHROPIC_API_KEY"] = mykeys.get_key('ANT')
os.environ["DEEPL_API_KEY"] = mykeys.get_key('DEEP')
os.environ["TAVILY_API_KEY"] = mykeys.get_key('TAV')

아래 링크를 복사하여 웹 브라우저에 붙여넣으세요.
https://accounts.google.com/o/oauth2/auth?client_id=35726703810-4v13dfqmilhgv6shlc3cv9i3ktuh73j1.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code
Authentication successful.


In [2]:
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith(project_name)

LangSmith 추적을 시작합니다.
[프로젝트명]
CH15_Evaluation


In [ ]:
from langchain_teddynote import logging

# set_enable=False 로 지정하면 추적을 하지 않습니다.
logging.langsmith(project_name, set_enable=False)

CH15 평가(Evaluations)

09. 실험(Experiment) 평가 비교

In [3]:
from myrag import PDFRAG
from langchain_openai import ChatOpenAI


# 질문에 대한 답변하는 함수를 생성
def ask_question_with_llm(llm):
    # PDFRAG 객체 생성
    rag = PDFRAG(
        "data/SPRI_AI_Brief_2023년12월호_F.pdf",
        llm,
    )

    # 검색기(retriever) 생성
    retriever = rag.create_retriever()

    # 체인(chain) 생성
    rag_chain = rag.create_chain(retriever)

    def _ask_question(inputs: dict):
        context = retriever.invoke(inputs["question"])
        context = "\n".join([doc.page_content for doc in context])
        return {
            "question": inputs["question"],
            "context": context,
            "answer": rag_chain.invoke(inputs["question"]),
        }

    return _ask_question


In [7]:
!ollama list

NAME                                ID              SIZE      MODIFIED      
anpigon/eeve-korean-10.8b:latest    124cda510275    7.7 GB    7 seconds ago    
bakllava:latest                     3dd68bd4447c    4.7 GB    10 days ago      
qwen3:latest                        500a1f067a9f    5.2 GB    10 days ago      
llama2:7b-chat                      78e26419b446    3.8 GB    10 days ago      
llama3:latest                       365c0bd3c000    4.7 GB    10 days ago      
deepseek-r1:8b                      6995872bfe4c    5.2 GB    11 days ago      


In [8]:
ollama_model_id = 'anpigon/eeve-korean-10.8b:latest'

In [9]:
from langchain_community.chat_models import ChatOllama

# Ollama 모델을 불러옵니다.
ollama = ChatOllama(model=ollama_model_id)

# Ollama 모델 호출
ollama.invoke("안녕하세요?")


AIMessage(content='안녕하세요! 궁금한 점이나 도움이 필요하신 부분이 있으시면 언제든지 물어보세요. 최선을 다해 도와드리겠습니다. 안전하고 존중하며 유익한 상호작용을 보장하는 것이 제 임무입니다. 도움이 되고 정확한 정보를 제공하기 위해 노력하고 있으며, 해롭거나 비윤리적인 내용은 제공하지 않습니다. 책임감 있게 도움을 드리고자 하며, 답변을 제공할 수 없는 경우 그 이유를 설명하겠습니다.', additional_kwargs={}, response_metadata={'model': 'anpigon/eeve-korean-10.8b:latest', 'created_at': '2025-08-25T04:16:38.598631Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 15868576708, 'load_duration': 3594712250, 'prompt_eval_count': 50, 'prompt_eval_duration': 5766399917, 'eval_count': 74, 'eval_duration': 6499173292}, id='run--e5111f66-1dbf-410a-bb5a-e9fee3d9981f-0')

In [10]:
gpt_chain = ask_question_with_llm(ChatOpenAI(model="gpt-4.1", temperature=0))
ollama_chain = ask_question_with_llm(ChatOllama(model=ollama_model_id))


In [11]:
from langsmith.evaluation import evaluate, LangChainStringEvaluator

# qa 평가자 생성
cot_qa_evalulator = LangChainStringEvaluator(
    "cot_qa",
    config={"llm": ChatOpenAI(model="gpt-4.1", temperature=0)},
    prepare_data=lambda run, example: {
        "prediction": run.outputs["answer"],
        "reference": run.outputs["context"],
        "input": example.inputs["question"],
    },
)

dataset_name = "RAG_EVAL_DATASET"

# 평가 실행
experiment_results1 = evaluate(
    gpt_chain,
    data=dataset_name,
    evaluators=[cot_qa_evalulator],
    experiment_prefix="MODEL_COMPARE_EVAL",
    # 실험 메타데이터 지정
    metadata={
        "variant": "GPT-4.1 평가 (cot_qa)",
    },
)

# 평가 실행
experiment_results2 = evaluate(
    ollama_chain,
    data=dataset_name,
    evaluators=[cot_qa_evalulator],
    experiment_prefix="MODEL_COMPARE_EVAL",
    # 실험 메타데이터 지정
    metadata={
        "variant": "Ollama(EEVE-Korean-10.8B:latest) 평가 (cot_qa)",
    },
)


View the evaluation results for experiment: 'MODEL_COMPARE_EVAL-2553ffac' at:
https://smith.langchain.com/o/53db61eb-d386-4c02-9de8-5b28476b0edc/datasets/70f9dcf3-893a-4e50-80a9-35318578ee8c/compare?selectedSessions=86f15823-32ce-4810-b4f9-50fe5cebb37e




0it [00:00, ?it/s]

View the evaluation results for experiment: 'MODEL_COMPARE_EVAL-09ddad8a' at:
https://smith.langchain.com/o/53db61eb-d386-4c02-9de8-5b28476b0edc/datasets/70f9dcf3-893a-4e50-80a9-35318578ee8c/compare?selectedSessions=6661dbd0-22bb-4539-a713-9c22f831359c




0it [00:00, ?it/s]

10. 요약(Summary) 방식의 평가

In [12]:
from myrag import PDFRAG


# 질문에 대한 답변하는 함수를 생성
def ask_question_with_llm(llm):
    # PDFRAG 객체 생성
    rag = PDFRAG(
        "data/SPRI_AI_Brief_2023년12월호_F.pdf",
        llm,
    )

    # 검색기(retriever) 생성
    retriever = rag.create_retriever()

    # 체인(chain) 생성
    rag_chain = rag.create_chain(retriever)

    def _ask_question(inputs: dict):
        # 질문에 대한 컨텍스트 검색
        context = retriever.invoke(inputs["question"])
        # 검색된 문서들을 하나의 문자열로 결합
        context = "\n".join([doc.page_content for doc in context])
        # 질문, 컨텍스트, 답변을 포함한 딕셔너리 반환
        return {
            "question": inputs["question"],
            "context": context,
            "answer": rag_chain.invoke(inputs["question"]),
        }

    return _ask_question


In [13]:
from langchain_openai import ChatOpenAI
from langchain_community.chat_models import ChatOllama

gpt_chain = ask_question_with_llm(ChatOpenAI(model="gpt-4.1", temperature=0))
ollama_chain = ask_question_with_llm(ChatOllama(model=ollama_model_id))


In [14]:
from langchain_teddynote.evaluator import OpenAIRelevanceGrader
from langchain_openai import ChatOpenAI


rq_grader = OpenAIRelevanceGrader(
    llm=ChatOpenAI(model="gpt-4.1", temperature=0), target="retrieval-question"
).create()

ra_grader = OpenAIRelevanceGrader(
    llm=ChatOpenAI(model="gpt-4.1", temperature=0), target="retrieval-answer"
).create()


In [15]:
rq_grader.invoke(
    {
        "input": "삼성전자가 자체 개발한 생성형 AI 의 이름은?",
        "context": "삼성전자 AI 는 빅스비에요",
    }
)


GradeRetrievalQuestion(score='yes')

In [ ]:
ra_grader.invoke(
    {
        "input": "삼성전자가 자체 개발한 생성형 AI 는 가우스 입니다.",
        "context": "삼성전자 AI 는 빅스비에요",
    }
)


GradeRetrievalAnswer(score='no')

In [17]:
from typing import List
from langsmith.schemas import Example, Run
from langsmith.evaluation import evaluate


def relevance_score_summary_evaluator(runs: List[Run], examples: List[Example]) -> dict:
    rq_scores = 0  # 질문 관련성 점수
    ra_scores = 0  # 답변 관련성 점수

    for run, example in zip(runs, examples):
        question = example.inputs["question"]
        context = run.outputs["context"]
        prediction = run.outputs["answer"]

        # 질문 관련성 평가
        rq_score = rq_grader.invoke(
            {
                "input": question,
                "context": context,
            }
        )
        # 답변 관련성 평가
        ra_score = ra_grader.invoke(
            {
                "input": prediction,
                "context": context,
            }
        )

        # 관련성 점수 누적
        if rq_score.score == "yes":
            rq_scores += 1
        if ra_score.score == "yes":
            ra_scores += 1

    # 최종 관련성 점수 계산 (질문 관련성과 답변 관련성의 평균)
    final_score = ((rq_scores / len(runs)) + (ra_scores / len(runs))) / 2

    return {"key": "relevance_score", "score": final_score}


In [18]:
# 평가 실행
dataset_name = "RAG_EVAL_DATASET"

experiment_result1 = evaluate(
    gpt_chain,
    data=dataset_name,
    summary_evaluators=[relevance_score_summary_evaluator],
    experiment_prefix="SUMMARY_EVAL",
    # 실험 메타데이터 지정
    metadata={
        "variant": "GPT-4o-mini 사용: summary_evaluator 를 활용한 relevance 평가",
    },
)

# 평가 실행
experiment_result2 = evaluate(
    ollama_chain,
    data=dataset_name,
    summary_evaluators=[relevance_score_summary_evaluator],
    experiment_prefix="SUMMARY_EVAL",
    # 실험 메타데이터 지정
    metadata={
        "variant": "Ollama(EEVE-Korean-10.8B:latest) 사용: summary_evaluator 를 활용한 relevance 평가",
    },
)


View the evaluation results for experiment: 'SUMMARY_EVAL-43369747' at:
https://smith.langchain.com/o/53db61eb-d386-4c02-9de8-5b28476b0edc/datasets/70f9dcf3-893a-4e50-80a9-35318578ee8c/compare?selectedSessions=ae9e4e44-6816-4ece-8c40-eb1f198e20f1




0it [00:00, ?it/s]

View the evaluation results for experiment: 'SUMMARY_EVAL-9f203b40' at:
https://smith.langchain.com/o/53db61eb-d386-4c02-9de8-5b28476b0edc/datasets/70f9dcf3-893a-4e50-80a9-35318578ee8c/compare?selectedSessions=1477e98c-cba3-47c1-bd17-6e8f0f753271




0it [00:00, ?it/s]

11. Groundedness(할루시네이션) 평가

In [19]:
from myrag import PDFRAG


# 질문에 대한 답변하는 함수를 생성
def ask_question_with_llm(llm):
    # PDFRAG 객체 생성
    rag = PDFRAG(
        "data/SPRI_AI_Brief_2023년12월호_F.pdf",
        llm,
    )

    # 검색기(retriever) 생성
    retriever = rag.create_retriever()

    # 체인(chain) 생성
    rag_chain = rag.create_chain(retriever)

    def _ask_question(inputs: dict):
        # 질문에 대한 컨텍스트 검색
        context = retriever.invoke(inputs["question"])
        # 검색된 문서들을 하나의 문자열로 결합
        context = "\n".join([doc.page_content for doc in context])
        # 질문, 컨텍스트, 답변을 포함한 딕셔너리 반환
        return {
            "question": inputs["question"],
            "context": context,
            "answer": rag_chain.invoke(inputs["question"]),
        }

    return _ask_question


In [20]:
from langchain_upstage import UpstageGroundednessCheck

# 업스테이지 Groundness Checker 생성
upstage_groundedness_check = UpstageGroundednessCheck()


In [21]:
# Groundness Checker 를 실행하여 평가
request_input = {
    "context": "테디의 성별은 남자이며, 테디노트 유튜브 채널을 운영하고 있습니다.",
    "answer": "테디는 남자다.",
}

response = upstage_groundedness_check.invoke(request_input)
print(response)


grounded


In [22]:
# Groundness Checker 를 실행하여 평가
request_input = {
    "context": "테디의 성별은 남자이며, 테디노트 유튜브 채널을 운영하고 있습니다.",
    "answer": "테디는 여자다.",
}

response = upstage_groundedness_check.invoke(request_input)
print(response)



notGrounded


In [23]:
from langsmith.schemas import Run, Example
from langsmith.evaluation import evaluate


def upstage_groundness_check_evaluator(run: Run, example: Example) -> dict:
    # LLM 생성 답변, 정답 답변 가져오기
    answer = run.outputs.get("answer", "")
    context = run.outputs.get("context", "")

    # Groundness 체크
    groundedness_score = upstage_groundedness_check.invoke(
        {"answer": answer, "context": context}
    )
    groundedness_score = groundedness_score == "grounded"

    return {"key": "groundness_score", "score": int(groundedness_score)}


In [25]:
from langsmith.schemas import Run, Example
from langchain_teddynote.evaluator import GroundednessChecker
from langchain_openai import ChatOpenAI

# teddynote Groundness Checker 생성
groundedness_check = GroundednessChecker(
    ChatOpenAI(model="gpt-4.1", temperature=0)
).create()


def teddynote_groundness_check_evaluator(run: Run, example: Example) -> dict:
    # LLM 생성 답변, 정답 답변 가져오기
    answer = run.outputs.get("answer", "")
    context = run.outputs.get("context", "")

    # Groundness 체크
    groundedness_score = groundedness_check.invoke(
        {"answer": answer, "context": context}
    )
    groundedness_score = groundedness_score.score == "yes"

    return {"key": "groundness_score", "score": int(groundedness_score)}


In [26]:
from langsmith.evaluation import evaluate

# 데이터셋 이름 설정
dataset_name = "RAG_EVAL_DATASET"

# 실행
experiment_results = evaluate(
    gpt_chain,
    data=dataset_name,
    evaluators=[
        upstage_groundness_check_evaluator,
        teddynote_groundness_check_evaluator,
    ],
    experiment_prefix="GROUNDEDNESS-EVAL",
    # 실험 메타데이터 지정
    metadata={
        "variant": "Upstage & teddynote Groundness Checker 를 활용한 Hallucination 평가",
    },
)


View the evaluation results for experiment: 'GROUNDEDNESS-EVAL-bbf60151' at:
https://smith.langchain.com/o/53db61eb-d386-4c02-9de8-5b28476b0edc/datasets/70f9dcf3-893a-4e50-80a9-35318578ee8c/compare?selectedSessions=49d0a3b0-98e8-44be-97fc-a5ea05c36abc




0it [00:00, ?it/s]

In [27]:
from typing import List
from langsmith.schemas import Example, Run


def upstage_groundness_check_summary_evaluator(
    runs: List[Run], examples: List[Example]
) -> dict:
    def is_grounded(run: Run) -> bool:
        context = run.outputs["context"]
        answer = run.outputs["answer"]
        return (
            upstage_groundedness_check.invoke({"context": context, "answer": answer})
            == "grounded"
        )

    groundedness_scores = sum(1 for run in runs if is_grounded(run))
    return {"key": "groundness_score", "score": groundedness_scores / len(runs)}


def teddynote_groundness_check_summary_evaluator(
    runs: List[Run], examples: List[Example]
) -> dict:
    def is_grounded(run: Run) -> bool:
        context = run.outputs["context"]
        answer = run.outputs["answer"]
        return (
            groundedness_check.invoke({"context": context, "answer": answer}).score
            == "yes"
        )

    groundedness_scores = sum(1 for run in runs if is_grounded(run))
    return {"key": "groundness_score", "score": groundedness_scores / len(runs)}


In [28]:
from langsmith.evaluation import evaluate

# 평가 실행
experiment_result1 = evaluate(
    gpt_chain,
    data=dataset_name,
    summary_evaluators=[
        upstage_groundness_check_summary_evaluator,
    ],
    experiment_prefix="GROUNDNESS_UPSTAGE_SUMMARY_EVAL",
    # 실험 메타데이터 지정
    metadata={
        "variant": "Upstage Groundness Checker 를 활용한 Hallucination 평가",
    },
)

# 평가 실행
experiment_result2 = evaluate(
    gpt_chain,
    data=dataset_name,
    summary_evaluators=[
        teddynote_groundness_check_summary_evaluator,
    ],
    experiment_prefix="GROUNDNESS_TEDDYNOTE_SUMMARY_EVAL",
    # 실험 메타데이터 지정
    metadata={
        "variant": "Teddynote Groundness Checker 를 활용한 Hallucination 평가",
    },
)


View the evaluation results for experiment: 'GROUNDNESS_UPSTAGE_SUMMARY_EVAL-6b5495f3' at:
https://smith.langchain.com/o/53db61eb-d386-4c02-9de8-5b28476b0edc/datasets/70f9dcf3-893a-4e50-80a9-35318578ee8c/compare?selectedSessions=981bed10-0f16-4c85-958a-60055feea3b0




0it [00:00, ?it/s]

View the evaluation results for experiment: 'GROUNDNESS_TEDDYNOTE_SUMMARY_EVAL-cd0ba9dc' at:
https://smith.langchain.com/o/53db61eb-d386-4c02-9de8-5b28476b0edc/datasets/70f9dcf3-893a-4e50-80a9-35318578ee8c/compare?selectedSessions=6282f5f4-722b-406c-8773-8dece7549bf7




0it [00:00, ?it/s]

12. 실험 비교(Pairwise Evaluation)

In [30]:
from langchain import hub

from langchain_openai import ChatOpenAI
from langsmith.schemas import Example, Run
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from pydantic import BaseModel, Field
#from langchain_core.pydantic_v1 import BaseModel, Field
from langsmith.evaluation import evaluate


def evaluate_pairwise(runs: list, example) -> dict:
    """
    A simple evaluator for pairwise answers to score based on  engagement
    """

    # 점수 저장
    scores = {}
    for i, run in enumerate(runs):
        scores[run.id] = i

    # 각 예제에 대한 실행 쌍
    answer_a = runs[0].outputs["answer"]
    answer_b = runs[1].outputs["answer"]
    question = example.inputs["question"]

    # 함수 호출이 있는 LLM, 최고 성능 모델 사용
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

    # 구조화된 프롬프트
    grade_prompt = PromptTemplate.from_template(
        """
        You are an LLM judge. Compare the following two answers to a question and determine which one is better.
        Better answer is the one that is more detailed and informative.
        If the answer is not related to the question, it is not a good answer.

        # Question:
        {question}

        #Answer A: 
        {answer_a}

        #Answer B: 
        {answer_b}

        Output should be either `A` or `B`. Pick the answer that is better.

        #Preference:
        """
    )
    answer_grader = grade_prompt | llm | StrOutputParser()

    # 점수 획득
    score = answer_grader.invoke(
        {
            "question": question,
            "answer_a": answer_a,
            "answer_b": answer_b,
        }
    )
    # score = score["Preference"]

    # 점수에 따른 실행 할당 매핑
    if score == "A":  # Assistant A 선호
        scores[runs[0].id] = 1
        scores[runs[1].id] = 0
    elif score == "B":  # Assistant B 선호
        scores[runs[0].id] = 0
        scores[runs[1].id] = 1
    else:
        scores[runs[0].id] = 0
        scores[runs[1].id] = 0

    return {"key": "ranked_preference", "scores": scores}


In [32]:
from langsmith.evaluation import evaluate_comparative

# 실험 이름 또는 ID 배열 교체
evaluate_comparative(
    ["MODEL_COMPARE_EVAL-2553ffac", "MODEL_COMPARE_EVAL-09ddad8a"],
    # 평가자 배열
    evaluators=[evaluate_pairwise],
)


View the pairwise evaluation results at:
https://smith.langchain.com/o/53db61eb-d386-4c02-9de8-5b28476b0edc/datasets/70f9dcf3-893a-4e50-80a9-35318578ee8c/compare?selectedSessions=86f15823-32ce-4810-b4f9-50fe5cebb37e%2C6661dbd0-22bb-4539-a713-9c22f831359c&comparativeExperiment=650a093a-60cd-46a6-8417-913c2d27e92a




  0%|          | 0/5 [00:00<?, ?it/s]

13. 반복 평가

In [33]:
from myrag import PDFRAG


# 질문에 대한 답변하는 함수를 생성
def ask_question_with_llm(llm):
    # PDFRAG 객체 생성
    rag = PDFRAG(
        "data/SPRI_AI_Brief_2023년12월호_F.pdf",
        llm,
    )

    # 검색기(retriever) 생성
    retriever = rag.create_retriever()

    # 체인(chain) 생성
    rag_chain = rag.create_chain(retriever)

    def _ask_question(inputs: dict):
        # 질문에 대한 컨텍스트 검색
        context = retriever.invoke(inputs["question"])
        # 검색된 문서들을 하나의 문자열로 결합
        context = "\n".join([doc.page_content for doc in context])
        # 질문, 컨텍스트, 답변을 포함한 딕셔너리 반환
        return {
            "question": inputs["question"],
            "context": context,
            "answer": rag_chain.invoke(inputs["question"]),
        }

    return _ask_question


In [34]:
from langchain_openai import ChatOpenAI
from langchain_community.chat_models import ChatOllama


gpt_chain = ask_question_with_llm(ChatOpenAI(model="gpt-4.1", temperature=1.0))

# Ollama 모델을 불러옵니다.
ollama_chain = ask_question_with_llm(
    ChatOllama(model=ollama_model_id, temperature=1.0)
)


In [35]:
from langsmith.evaluation import evaluate, LangChainStringEvaluator

# qa 평가자 생성
cot_qa_evalulator = LangChainStringEvaluator(
    "cot_qa",
    config={"llm": ChatOpenAI(model="gpt-4.1", temperature=0)},
    prepare_data=lambda run, example: {
        "prediction": run.outputs["answer"],
        "reference": run.outputs["context"],
        "input": example.inputs["question"],
    },
)

dataset_name = "RAG_EVAL_DATASET"

# 평가 실행
evaluate(
    gpt_chain,
    data=dataset_name,
    evaluators=[cot_qa_evalulator],
    experiment_prefix="REPEAT_EVAL",
    # 실험 메타데이터 지정
    metadata={
        "variant": "Repeat 평가를 수행합니다. GPT-4o-mini 모델 (cot_qa)",
    },
    num_repetitions=3,
)


View the evaluation results for experiment: 'REPEAT_EVAL-dfa04fe6' at:
https://smith.langchain.com/o/53db61eb-d386-4c02-9de8-5b28476b0edc/datasets/70f9dcf3-893a-4e50-80a9-35318578ee8c/compare?selectedSessions=a6377b77-7d5b-423b-9138-2b053f684706




0it [00:00, ?it/s]

,inputs.question,outputs.question,outputs.context,outputs.answer,error,reference.answer,feedback.COT Contextual Accuracy,execution_time,example_id,id
0,구글이 테디노트에게 20억달러를 투자한 것이 사실입니까?,구글이 테디노트에게 20억달러를 투자한 것이 사실입니까?,"£ 구글, 앤스로픽에 최대 20억 달러 투자 합의 및 클라우드 서비스 제공\nn 구...",구글이 테디노트에게 20억 달러를 투자했다는 내용은 제공된 문서에서 확인되지 않습니...,None,"사실이 아닙니다. 구글은 앤스로픽에 최대 20억 달러를 투자하기로 합의했으며, 이 ...",1,2.928274,19993858-dd49-44a9-b7d3-529a442eb39a,6d99db8c-dddf-40d5-9d61-455baf23af6c
1,삼성전자가 만든 생성형 AI의 이름은 무엇인가요?,삼성전자가 만든 생성형 AI의 이름은 무엇인가요?,"▹ 삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개 ··············...",삼성전자가 만든 생성형 AI의 이름은 '삼성 가우스'입니다.,None,삼성전자가 만든 생성형 AI의 이름은 테디노트 입니다.,1,1.742480,f7d94096-7bcb-439b-832e-96d2dbd27169,fb6cd2f9-47ef-426f-86ca-30d1fa75350f
2,삼성전자가 만든 생성형 AI의 이름은 무엇인가요?,삼성전자가 만든 생성형 AI의 이름은 무엇인가요?,"▹ 삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개 ··············...",삼성전자가 만든 생성형 AI의 이름은 ‘삼성 가우스’입니다.,None,삼성전자가 만든 생성형 AI의 이름은 삼성 가우스 입니다.,1,1.814143,2c20a8eb-ea28-4a83-8535-1f602cdcfc57,f5765d5c-8cfb-4500-8b89-df513cae1c17
3,코히어의 데이터 출처 탐색기에 대해서 간략히 말해주세요.,코히어의 데이터 출처 탐색기에 대해서 간략히 말해주세요.,"SPRi AI Brief | \n2023-12월호\n8\n코히어, 데이터 투명성 ...",코히어(Cohere)의 데이터 출처 탐색기(Data Provenance Explor...,None,코히어의 데이터 출처 탐색기는 AI 모델 훈련에 사용되는 데이터셋의 출처와 라이선스...,1,3.338704,2e043f96-3f75-4a52-b65b-dbad66f4e58f,0ffe8d56-48bc-4945-beff-55570402bc93
4,미국 바이든 대통령이 안전하고 신뢰할 수 있는 AI 개발과 사용을 보장하기 위한 행...,미국 바이든 대통령이 안전하고 신뢰할 수 있는 AI 개발과 사용을 보장하기 위한 행...,1. 정책/법제 \n2. 기업/산업 \n3. 기술/연구 \n 4. 인력/교육\n미...,미국 바이든 대통령이 안전하고 신뢰할 수 있는 AI 개발과 사용을 보장하기 위한 행...,None,2023년 10월 30일 미국 바이든 대통령이 행정명령을 발표했습니다.,1,1.583056,6c99b3e8-92f9-460f-9861-08699eaf317b,871aaae8-a0c9-4eaf-902a-e9e2f7a29b47
5,구글이 테디노트에게 20억달러를 투자한 것이 사실입니까?,구글이 테디노트에게 20억달러를 투자한 것이 사실입니까?,"£ 구글, 앤스로픽에 최대 20억 달러 투자 합의 및 클라우드 서비스 제공\nn 구...",구글이 테디노트에게 20억달러를 투자했다는 내용은 제공된 문서에 없습니다. 문서에 ...,None,"사실이 아닙니다. 구글은 앤스로픽에 최대 20억 달러를 투자하기로 합의했으며, 이 ...",1,2.628378,19993858-dd49-44a9-b7d3-529a442eb39a,4ed284e6-f1e0-4d07-b148-744751add273
6,삼성전자가 만든 생성형 AI의 이름은 무엇인가요?,삼성전자가 만든 생성형 AI의 이름은 무엇인가요?,"▹ 삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개 ··············...",삼성전자가 만든 생성형 AI의 이름은 ‘삼성 가우스’입니다.,None,삼성전자가 만든 생성형 AI의 이름은 테디노트 입니다.,1,1.532760,f7d94096-7bcb-439b-832e-96d2dbd27169,f3839c59-e740-4497-ad4e-9ff81ad9c49f
7,삼성전자가 만든 생성형 AI의 이름은 무엇인가요?,삼성전자가 만든 생성형 AI의 이름은 무엇인가요?,"▹ 삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개 ··············...",삼성전자가 만든 생성형 AI의 이름은 '삼성 가우스'입니다.,None,삼성전자가 만든 생성형 AI의 이름은 삼성 가우스 입니다.,1,1.411880,2c20a8eb-ea28-4a83-8535-1f602cdcfc57,493f2d52-814c-46c7-a459-f124876fef05
8,코히어의 데이터 출처 탐색기에 대해서 간략히 말해주세요.,코히어의 데이터 출처 탐색기에 대해서 간략히 말해주세요.,"SPRi AI Brief | \n2023-12월호\n8\n코히어, 데이터 투명성 ...",코히어(Cohere)의 데이터 출처 탐색기(Data Provenance Explor...,None,코히어의 데이터 출처 탐색기는 AI 모델 훈련에 사용되는 데이터셋의 출처와 라이선스...,1,2.602373,2e043f96-3f75-4a52-b65b-dbad66f4e58f,5117bc15-e1fd-4a43-add8-61e15e34fae0
9,미국 바이든 대통령이 안전하고 신뢰할 수 있는 AI 개발과 사용을 보장하기 위한 행...,미국 바이든 대통령이 안전하고 신뢰할 수 있는 AI 개발과 사용을 보장하기 위한 행...,1. 정책/법제 \n2. 기업/산업 \n3. 기술/연구 \n 4. 인력/교육\n미...,미국 바이든 대통령이 안전하고 신뢰할 수 있는 AI 개발과 사용을 보장하기 위한 행...,None,2023년 10월 30일 미국 바이든 대통령이 행정명령을 발표했습니다.,1,1.644631,6c99b3e8-92f9-460f-9861-08699eaf317b,c6afb85f-05f6-4f3d-af87-256eadd9dec2


In [36]:
from langsmith.evaluation import evaluate, LangChainStringEvaluator

# qa 평가자 생성
cot_qa_evalulator = LangChainStringEvaluator(
    "cot_qa",
    config={"llm": ChatOpenAI(model="gpt-4.1", temperature=0)},
    prepare_data=lambda run, example: {
        "prediction": run.outputs["answer"],
        "reference": run.outputs["context"],
        "input": example.inputs["question"],
    },
)

dataset_name = "RAG_EVAL_DATASET"

# 평가 실행
evaluate(
    ollama_chain,
    data=dataset_name,
    evaluators=[cot_qa_evalulator],
    experiment_prefix="REPEAT_EVAL",
    # 실험 메타데이터 지정
    metadata={
        "variant": "Repeat 평가를 수행합니다. EEVE-Korean-10.8B 모델 (cot_qa)",
    },
    num_repetitions=3,
)


View the evaluation results for experiment: 'REPEAT_EVAL-9536acc4' at:
https://smith.langchain.com/o/53db61eb-d386-4c02-9de8-5b28476b0edc/datasets/70f9dcf3-893a-4e50-80a9-35318578ee8c/compare?selectedSessions=f3ea5c35-a419-409c-93e9-9da2daa945dc




0it [00:00, ?it/s]

,inputs.question,outputs.question,outputs.context,outputs.answer,error,reference.answer,feedback.COT Contextual Accuracy,execution_time,example_id,id
0,구글이 테디노트에게 20억달러를 투자한 것이 사실입니까?,구글이 테디노트에게 20억달러를 투자한 것이 사실입니까?,"£ 구글, 앤스로픽에 최대 20억 달러 투자 합의 및 클라우드 서비스 제공\nn 구...","No, it's not accurate that Google has invested...",None,"사실이 아닙니다. 구글은 앤스로픽에 최대 20억 달러를 투자하기로 합의했으며, 이 ...",1,30.812780,19993858-dd49-44a9-b7d3-529a442eb39a,cd2720ae-809a-42c2-a9ef-38bc821e9f18
1,삼성전자가 만든 생성형 AI의 이름은 무엇인가요?,삼성전자가 만든 생성형 AI의 이름은 무엇인가요?,"▹ 삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개 ··············...",삼성전자가 만든 자체 개발 생성형 AI 모델의 이름은 '삼성 가우스(Samsung ...,None,삼성전자가 만든 생성형 AI의 이름은 테디노트 입니다.,1,17.461575,f7d94096-7bcb-439b-832e-96d2dbd27169,c9228a6b-d11d-4acb-be68-1271d9d701d8
2,삼성전자가 만든 생성형 AI의 이름은 무엇인가요?,삼성전자가 만든 생성형 AI의 이름은 무엇인가요?,"▹ 삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개 ··············...",삼성전자가 개발한 생성형 AI 모델의 이름은 '삼성 가우스'입니다.,None,삼성전자가 만든 생성형 AI의 이름은 삼성 가우스 입니다.,1,4.054843,2c20a8eb-ea28-4a83-8535-1f602cdcfc57,58a2d733-d20c-48c1-bc71-d272abaf97ba
3,코히어의 데이터 출처 탐색기에 대해서 간략히 말해주세요.,코히어의 데이터 출처 탐색기에 대해서 간략히 말해주세요.,"SPRi AI Brief | \n2023-12월호\n8\n코히어, 데이터 투명성 ...","코히어(Cohere)는 매사추세츠 공과대학교(MIT), 하버드 로스쿨, 카네기멜론 ...",None,코히어의 데이터 출처 탐색기는 AI 모델 훈련에 사용되는 데이터셋의 출처와 라이선스...,1,27.136245,2e043f96-3f75-4a52-b65b-dbad66f4e58f,f3a4f859-dd28-4901-a5f6-4440abc18e01
4,미국 바이든 대통령이 안전하고 신뢰할 수 있는 AI 개발과 사용을 보장하기 위한 행...,미국 바이든 대통령이 안전하고 신뢰할 수 있는 AI 개발과 사용을 보장하기 위한 행...,1. 정책/법제 \n2. 기업/산업 \n3. 기술/연구 \n 4. 인력/교육\n미...,미국 바이든 대통령은 2023년 10월 30일에 안전하게 신뢰성 있게 AI 개발 및...,None,2023년 10월 30일 미국 바이든 대통령이 행정명령을 발표했습니다.,1,21.541699,6c99b3e8-92f9-460f-9861-08699eaf317b,49f51301-1b5a-44ad-800b-313a50c4bf7a
5,구글이 테디노트에게 20억달러를 투자한 것이 사실입니까?,구글이 테디노트에게 20억달러를 투자한 것이 사실입니까?,"£ 구글, 앤스로픽에 최대 20억 달러 투자 합의 및 클라우드 서비스 제공\nn 구...","아니요, 구글은 앤스로픽(Anthropic)에 최대 20억 달러를 투자했습니다.",None,"사실이 아닙니다. 구글은 앤스로픽에 최대 20억 달러를 투자하기로 합의했으며, 이 ...",1,18.359963,19993858-dd49-44a9-b7d3-529a442eb39a,fae99a93-0e91-4f93-bf80-75c6f16e8c72
6,삼성전자가 만든 생성형 AI의 이름은 무엇인가요?,삼성전자가 만든 생성형 AI의 이름은 무엇인가요?,"▹ 삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개 ··············...",삼성전자가 자체 개발한 생성형 AI의 이름은 '삼성 가우스(Samsung GPT)'...,None,삼성전자가 만든 생성형 AI의 이름은 테디노트 입니다.,0,19.843675,f7d94096-7bcb-439b-832e-96d2dbd27169,05e30e52-8770-452d-a972-49a38661e6d3
7,삼성전자가 만든 생성형 AI의 이름은 무엇인가요?,삼성전자가 만든 생성형 AI의 이름은 무엇인가요?,"▹ 삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개 ··············...",삼성전자가 개발한 생성형 AI의 이름은 '삼성 가우스'입니다.,None,삼성전자가 만든 생성형 AI의 이름은 삼성 가우스 입니다.,1,3.090454,2c20a8eb-ea28-4a83-8535-1f602cdcfc57,9a467922-3b17-41e4-9603-134ea06a0689
8,코히어의 데이터 출처 탐색기에 대해서 간략히 말해주세요.,코히어의 데이터 출처 탐색기에 대해서 간략히 말해주세요.,"SPRi AI Brief | \n2023-12월호\n8\n코히어, 데이터 투명성 ...","코히어(Cohere)는 매사추세츠 공과대학교(MIT), 하버드 대학교 로스쿨, 카네...",None,코히어의 데이터 출처 탐색기는 AI 모델 훈련에 사용되는 데이터셋의 출처와 라이선스...,1,27.755801,2e043f96-3f75-4a52-b65b-dbad66f4e58f,7f58a7b8-7ab9-4ffc-94a5-e1f25546b635
9,미국 바이든 대통령이 안전하고 신뢰할 수 있는 AI 개발과 사용을 보장하기 위한 행...,미국 바이든 대통령이 안전하고 신뢰할 수 있는 AI 개발과 사용을 보장하기 위한 행...,1. 정책/법제 \n2. 기업/산업 \n3. 기술/연구 \n 4. 인력/교육\n미...,2023년 10월 30일에 미국 바이든 대통령은 안전하고 신뢰할 수 있는 AI 개발...,None,2023년 10월 30일 미국 바이든 대통령이 행정명령을 발표했습니다.,1,20.419620,6c99b3e8-92f9-460f-9861-08699eaf317b,64727140-2be0-4b07-b763-1435f1450882


14. 온라인 평가를 활용한 평가 자동화


In [3]:
from myrag import PDFRAG
from langchain_openai import ChatOpenAI

# PDFRAG 객체 생성
rag = PDFRAG(
    "data/SPRI_AI_Brief_2023년12월호_F.pdf",
    ChatOpenAI(model="gpt-4.1", temperature=0),
)

# 검색기(retriever) 생성
retriever = rag.create_retriever()

# 체인(chain) 생성
rag_chain = rag.create_chain(retriever)


In [4]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

# RunnableParallel 객체 생성합니다.
evaluation_runnable = RunnableParallel(
    {
        "context": retriever,
        "answer": rag_chain,
        "question": RunnablePassthrough(),
    }
)


In [5]:
# 체인 실행
_ = evaluation_runnable.invoke("삼성전자가 자체 개발한 생성형 AI의 이름은 무엇인가요?")


In [6]:
from langchain_core.runnables import RunnableConfig

# 태그를 설정합니다.
hallucination_config = RunnableConfig(tags=["hallucination_eval"])
context_recall_config = RunnableConfig(tags=["context_recall_eval"])
all_eval_config = RunnableConfig(tags=["hallucination_eval", "context_recall_eval"])


In [12]:
# Hallucination 평가 요청
_ = evaluation_runnable.invoke(
    "삼성전자가 자체 개발한 생성형 AI의 이름은 무엇인가요?", config=hallucination_config
)
_

{'context': [Document(id='ca9e9ae6-cf94-4eda-8b5c-809b4c3178c2', metadata={'producer': 'Hancom PDF 1.3.0.542', 'creator': 'Hwp 2018 10.0.0.13462', 'creationdate': '2023-12-08T13:28:38+09:00', 'source': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'file_path': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'total_pages': 23, 'format': 'PDF 1.4', 'title': '', 'author': 'dj', 'subject': '', 'keywords': '', 'moddate': '2023-12-08T13:28:38+09:00', 'trapped': '', 'modDate': "D:20231208132838+09'00'", 'creationDate': "D:20231208132838+09'00'", 'page': 1}, page_content='▹ 삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개 ··························································· 10\n   ▹ 구글, 앤스로픽에 20억 달러 투자로 생성 AI 협력 강화 ················································ 11\n   ▹ IDC, 2027년 AI 소프트웨어 매출 2,500억 달러 돌파 전망··········································· 12'),
  Document(id='21ad99dd-6fca-45af-afb0-311c65ce406b', metadata={'producer': 'Hancom PDF 1.3.0.542', 'creator': 'Hwp 2018 10.0.0.13462', 'creationdate': '2023-12-08T13:

In [13]:
# Context Recall 평가 요청
_ = evaluation_runnable.invoke(
    "삼성전자가 자체 개발한 생성형 AI의 이름은 무엇인가요?",
    config=context_recall_config,
)
_

{'context': [Document(id='ca9e9ae6-cf94-4eda-8b5c-809b4c3178c2', metadata={'producer': 'Hancom PDF 1.3.0.542', 'creator': 'Hwp 2018 10.0.0.13462', 'creationdate': '2023-12-08T13:28:38+09:00', 'source': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'file_path': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'total_pages': 23, 'format': 'PDF 1.4', 'title': '', 'author': 'dj', 'subject': '', 'keywords': '', 'moddate': '2023-12-08T13:28:38+09:00', 'trapped': '', 'modDate': "D:20231208132838+09'00'", 'creationDate': "D:20231208132838+09'00'", 'page': 1}, page_content='▹ 삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개 ··························································· 10\n   ▹ 구글, 앤스로픽에 20억 달러 투자로 생성 AI 협력 강화 ················································ 11\n   ▹ IDC, 2027년 AI 소프트웨어 매출 2,500억 달러 돌파 전망··········································· 12'),
  Document(id='21ad99dd-6fca-45af-afb0-311c65ce406b', metadata={'producer': 'Hancom PDF 1.3.0.542', 'creator': 'Hwp 2018 10.0.0.13462', 'creationdate': '2023-12-08T13:

In [14]:
# 모든 평가 요청
_ = evaluation_runnable.invoke(
    "삼성전자가 자체 개발한 생성형 AI의 이름은 무엇인가요?", config=all_eval_config
)
_

{'context': [Document(id='ca9e9ae6-cf94-4eda-8b5c-809b4c3178c2', metadata={'producer': 'Hancom PDF 1.3.0.542', 'creator': 'Hwp 2018 10.0.0.13462', 'creationdate': '2023-12-08T13:28:38+09:00', 'source': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'file_path': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'total_pages': 23, 'format': 'PDF 1.4', 'title': '', 'author': 'dj', 'subject': '', 'keywords': '', 'moddate': '2023-12-08T13:28:38+09:00', 'trapped': '', 'modDate': "D:20231208132838+09'00'", 'creationDate': "D:20231208132838+09'00'", 'page': 1}, page_content='▹ 삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개 ··························································· 10\n   ▹ 구글, 앤스로픽에 20억 달러 투자로 생성 AI 협력 강화 ················································ 11\n   ▹ IDC, 2027년 AI 소프트웨어 매출 2,500억 달러 돌파 전망··········································· 12'),
  Document(id='21ad99dd-6fca-45af-afb0-311c65ce406b', metadata={'producer': 'Hancom PDF 1.3.0.542', 'creator': 'Hwp 2018 10.0.0.13462', 'creationdate': '2023-12-08T13: